In [52]:
# 독립변수를 늘렸을때 어떻게 되는지 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error # 제곱해서 평균낸 잔차


from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공선성체크 (분산팽창요인이용)
import statsmodels.api as sm # statsmodels = 통계모델 의 여러가지 통계치를 보고 독립변수를 체크하려고씀

In [53]:
df = pd.read_csv("./data/cruise_ship_info.csv")

In [54]:
# df["Cruise_line"] => 이거 범주화를 숫자로 변경하는거 필요함
ohe = OneHotEncoder()
arr = np.array(df["Cruise_line"])
arr = np.reshape(arr, (-1,1))

cruise_line_name = ohe.fit_transform(arr)
ohe.get_feature_names_out() # 확인해보기

ohe_df = pd.DataFrame(
    cruise_line_name.toarray(),
    columns = ohe.get_feature_names_out()
)

ohe_df = pd.concat([
    df,
    ohe_df
], axis = 1)
ohe_df.head()

,Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew,x0_Azamara,...,x0_Oceania,x0_Orient,x0_P&O,x0_Princess,x0_Regent_Seven_Seas,x0_Royal_Caribbean,x0_Seabourn,x0_Silversea,x0_Star,x0_Windstar
0,Journey,Azamara,6,30.277,6.94,5.94,3.55,42.64,3.55,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Quest,Azamara,6,30.277,6.94,5.94,3.55,42.64,3.55,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Celebration,Carnival,26,47.262,14.86,7.22,7.43,31.80,6.70,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Conquest,Carnival,11,110.000,29.74,9.53,14.88,36.99,19.10,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Destiny,Carnival,17,101.353,26.42,8.92,13.21,38.36,10.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
# 필요없는 열 삭제
ohe_df = ohe_df.drop(["Ship_name","Cruise_line","Age","cabins"], axis = 1)
ohe_df.head()

,Tonnage,passengers,length,passenger_density,crew,x0_Azamara,x0_Carnival,x0_Celebrity,x0_Costa,x0_Crystal,...,x0_Oceania,x0_Orient,x0_P&O,x0_Princess,x0_Regent_Seven_Seas,x0_Royal_Caribbean,x0_Seabourn,x0_Silversea,x0_Star,x0_Windstar
0,30.277,6.94,5.94,42.64,3.55,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30.277,6.94,5.94,42.64,3.55,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,47.262,14.86,7.22,31.80,6.70,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,110.000,29.74,9.53,36.99,19.10,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,101.353,26.42,8.92,38.36,10.00,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
x = ohe_df.drop("crew", axis = 1) # 일단 다 넣어
y = df["crew"]

In [57]:
# 데이터 분할해 
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size = 0.25,
    random_state = 31
)

In [58]:
pf = PolynomialFeatures(include_bias = False)
poly_train = pf.fit_transform(x_train)
poly_test = pf.fit_transform(x_test)

In [59]:
lr_org = LinearRegression()
lr_org.fit(x_train, y_train)
print(lr_org.score(x_train,y_train))
print(lr_org.score(x_test,y_test))

0.9091976508070959
0.9471997715490457


In [51]:
y_pred = lr_org.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
mse

2.965134996070227